In [133]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"physical_activity_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

# EDIT THE VALUES IN THE CELL ABOVE THEN PRESS RUN ALL

In [134]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


In [135]:
DATA_DIRECTORY = "./all_respeck"
RESPIRATORY_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    "standing&coughing",
    "standing&hyperventilating",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating"
}

In [136]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in RESPIRATORY_ACTIVITIES:
        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [137]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [138]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(RESPIRATORY_ACTIVITIES)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

there are 14892 tagged sequences in the dataset


In [139]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        # layers.Conv1D(64, 3, activation='relu'),
        # layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [140]:

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test

# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, RESPIRATORY_ACTIVITIES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/Task2/{MODEL_NAME}_{test_accuracy}")

Epoch 1/50
1192/1192 [==============================] - 6s 4ms/step - loss: 1.0801 - accuracy: 0.4335 - val_loss: 0.9679 - val_accuracy: 0.4607
Epoch 2/50
1192/1192 [==============================] - 5s 4ms/step - loss: 0.8867 - accuracy: 0.5620 - val_loss: 0.7143 - val_accuracy: 0.6776
Epoch 3/50
1192/1192 [==============================] - 5s 4ms/step - loss: 0.6379 - accuracy: 0.7342 - val_loss: 0.8096 - val_accuracy: 0.6676
Epoch 4/50
1192/1192 [==============================] - 5s 4ms/step - loss: 0.5566 - accuracy: 0.7653 - val_loss: 0.5092 - val_accuracy: 0.7790
Epoch 5/50
1192/1192 [==============================] - 5s 4ms/step - loss: 0.5193 - accuracy: 0.7810 - val_loss: 0.4666 - val_accuracy: 0.7972
Epoch 6/50
1192/1192 [==============================] - 5s 4ms/step - loss: 0.4886 - accuracy: 0.7927 - val_loss: 0.4481 - val_accuracy: 0.8052
Epoch 7/50
1192/1192 [==============================] - 5s 4ms/step - loss: 0.4644 - accuracy: 0.8042 - val_loss: 0.4683 - val_accuracy:

2023-11-03 19:42:45.282606: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,10,128]
	 [[{{node inputs}}]]
2023-11-03 19:42:45.590989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,10,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/Task2/physical_activity_epochs:50_batch:10_gyro:False_window:4_overlap:3.keras_0.9147651195526123/assets


INFO:tensorflow:Assets written to: models/Task2/physical_activity_epochs:50_batch:10_gyro:False_window:4_overlap:3.keras_0.9147651195526123/assets
